In [ ]:
#hide
from fastbook import *
setup_book()

# A Language Model from Scratch

## The Data

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
# lines is a list of lines with a number on each line, ending in '\n'
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

In [ ]:
# we use a . to represent newline
text = ' . '.join([l.strip() for l in lines])
text[:100]

In [ ]:
# per-word tokenization
tokens = text.split(' ')
tokens[:10]

In [ ]:
# getting the vocab
vocab = L(*tokens).unique()
vocab

In [ ]:
# numericalization; see https://docs.python.org/3/library/functions.html#enumerate
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

## Our First Language Model from Scratch

In [ ]:
# first attempt: trigram model with Markov assumption
# example from data
# note: the data is not cut up very well; it gives
# ((0, 1, 2), 3); ((3, 4, 5), 6); ((6, 7, 8), 9)
# rather than
# ((0, 1, 2), 3); ((1, 2, 3), 4); ((2, 3, 4), 5)
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

In [ ]:
# create a list of tuples (we use tensor for input data)
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

In [ ]:
# create a dataloader. The sequential information of the input data is significant, hence shuffle=False
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=bs, shuffle=False)

### Our Language Model in PyTorch

In [ ]:
class LMModel1(Module):
    # An embedding layer [vocab_sz] -> [n_hidden]
    # A linear layer [n_hidden] -> [n_hidden]
    # A linear layer [n_hidden] -> [vocab_sz]
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        # self.i_h interprets the first word of the trigrams across the training data x[:,0] as one-hot vectors, then returns the [n_hidden] activations
        # self.h_h takes those activations and returns new activations of the same shape
        # then perform relu on those activations
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        # then add the embedding activations for the second word of the trigram
        h = h + self.i_h(x[:,1])
        # then pass through the same linear layer and then relu
        h = F.relu(self.h_h(h))
        # then add the embedding activations for the third word of the trigram
        # observe that in predicting the output d given (a, b, c); the contribution of (the embedding of) c to the 2nd-last layer is direct,
        # whereas the contribution of (the embeddings of) a, b is mediated through the state h
        h = h + self.i_h(x[:,2])
        # then pass through the same linear layer and then relu
        h = F.relu(self.h_h(h))
        # finally pass through the output layer
        return self.h_o(h)
    # the way we pass through the same linear layer thrice gives rise to why we call this an RNN.

In [ ]:
# we train this on the honestly defectively processed data we have;
# and observe that we can train it!
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

In [ ]:
# as a sanity check, we extract the most commonly predicted word on the validation set
# counts initialized to a vector corresponding to the vocab
# `range_of` is provided by fastcore
# recall that dls.valid, on each loop, yields (x, y) whese x: [bs, 3] and y: [bs]
# hence we are adding the number of labels every loop to n
# and for eacn vocab element, we specialize the #bs predictions to indicate whether we
# should predict that vocab item, and sum these indicators
# finally, we find the index of the most predicted element,
# display it, and also compute our accuracy if we had just predicted it.

# note that the text draws a defective conclusion:
# the separator is indeed the most common token if we include the training set
# it is the validation set that is interesting in that it starts well into the thousands
# so every number includes a thousand word, and it is a percularity of our tokenization that the
# last line does not terminate in '.' due to the use of the python `join` method
n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

### Our First Recurrent Neural Network

In [ ]:
# this cleans up the recurrence explicitly given earlier using a for loop
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

## Improving the RNN

### Maintaining the State of an RNN

In [ ]:
# h is maintained
# self.h.detach() resets the history of calculations h participates in for the purpose of gradient calculation
# hence only the last 3 layers will be considered
# when we backprop on a layer that is called multiple times, we call it backpropagation through time (BPTT)
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            # at the first execution of forward, at the first iteration of this loop,
            # self.h goes from a literal 0 (to be broadcasted) to being a vector of length bs
            self.h = self.h + self.i_h(x[:,i])
            # then at this step, even at the first iteration, self.h is already of the correct shape
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

In [ ]:
# we now discuss how to properly format the dataset for input in RNNs
# split the dataset into m batches
# recall that for an RNN, each batch coordinate-wise follows the previous batch, so
# that the ith batch is a vector of the (i modulo m)-th tokens
m = len(seqs)//bs
m,bs,len(seqs)

In [ ]:
# in general
# set m to the number of batches
# organize each batch as (i, i + m, i + 2m, ..., i + (bs-1)m), and note
# that later batches may be one entry shorter.
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [ ]:
# split the dataset into train and valid
# the argument for train= is
#   a list of length m
#   consisting of vectors of length bs
#   where each item is a tuple (x, y)
#   where x is a tensor of 3 numericalized words and y is the numericalized word to predict
# each batch of dls is a tuple (x, y) where x is [bs] x 3 and y is [bs]
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

In [ ]:
# ModelResetter resets the state after every relevant step
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

### Creating More Signal

In [ ]:
# recall our earlier remark that "the data is not cut up very well"
#   we cut the data up properly this time to make better use of the
#   sequential nature of our data.
# seqs is now a vector of pairs of 16-token sequences.
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [ ]:
[L(vocab[o] for o in s) for s in seqs[0]]

In [ ]:
# note that `sl` comes from the global scope in this definition
#   this time given (a, b, c, ..., p) we predict (b', c', ..., p', q')
#   using only e.g. b and the state to predict c'
# observe that when we maintained the state in LMModel3, we always advanced
# the context by 3 symbols before predicting the 4th symbol. But of coures
# a model that can only predict every 3rd letter is of limited utility.
# we want to predict as many words as is fed in (discounting stats).

class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        # stack the bs-length vectors ([bs] x [vocab_sz]) in the minor dimension (dim=1),
        # so that the major dimension remains of size bs, having shape [bs] x [sl] x [vocab_sz]
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [ ]:
# our outputs have shape [bs] x [sl] x [vocab_sz]; our targets have shape [bs] x [sl],
# and we flatten them to obtain the correct correspondence
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [ ]:
# performance in this RNN architecture is very variable since it suffers from gradient explosion
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

## Multilayer RNNs

### The Model

In [ ]:
# the recurrent linear layer in the previous model has an efficient implementation in torch as RNN
# and this also allows for multilayer RNNs where the output of one layer is passed to the next
# note that with these parameters, tanh is used as the nonlinearity instead of ReLU
# batch_first assumes that inputs ase organized as [bs] x [sl] x [n_hidden] rather than as [sl] x [bs] x [n_hidden]
# also note the shape of our state, and its dependence on the global `bs`.
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [ ]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

### Exploding or Disappearing Activations

## LSTM

### Building an LSTM from Scratch

In [ ]:
# stuff I already know
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.forget_gate = nn.Linear(ni + nh, nh)
        self.input_gate  = nn.Linear(ni + nh, nh)
        self.cell_gate   = nn.Linear(ni + nh, nh)
        self.output_gate = nn.Linear(ni + nh, nh)

    def forward(self, input, state):
        h,c = state
        h = torch.cat([h, input], dim=1)
        forget = torch.sigmoid(self.forget_gate(h))
        c = c * forget
        inp = torch.sigmoid(self.input_gate(h))
        cell = torch.tanh(self.cell_gate(h))
        c = c + inp * cell
        out = torch.sigmoid(self.output_gate(h))
        h = out * torch.tanh(c)
        return h, (h,c)

In [ ]:
# refactored
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)

    def forward(self, input, state):
        h,c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate,forgetgate,outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate*c) + (ingate*cellgate)
        h = outgate * c.tanh()
        return h, (h,c)

In [ ]:
# discussion on what the `chunk` function above does
t = torch.arange(0,10); t

In [ ]:
t.chunk(2)

### Training a Language Model Using LSTMs

In [ ]:
# we have here replaced the multilayer RNN with a multilayer LSTM
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

## Regularizing an LSTM

### Dropout

In [ ]:
# complementing a dropout layer
# during dropout, we must also normalize (on average) against the fraction of nodes dropped
class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

### Activation Regularization and Temporal Activation Regularization

Recall weight decay / L2 regularization, where large weights were penalized heavier on their errors
- In activation regularization, we directly penalize large activations (the h parameter / state)
- In temporal activation regularization, we accumulate the activations for the whole sequence, then penalize the difference between the activations for the sequence but the last with the activations for the sequence but the first.

### Training a Weight-Tied Regularized LSTM

By a weight-tied LSTM we use the same weights for the embedding layer mapping one-hot activations to features, as in the final layer mapping the activations (of the same shape as the features) to the vocabulary probability distribution.

In [ ]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel7(len(vocab), 64, 2, 0.5),
                loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                cbs=[ModelResetter, RNNRegularizer(alpha=2, beta=1)])

In [ ]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

## Conclusion

## Questionnaire

1. If the dataset for your project is so big and complicated that working with it takes a significant amount of time, what should you do?
1. Why do we concatenate the documents in our dataset before creating a language model?
1. To use a standard fully connected network to predict the fourth word given the previous three words, what two tweaks do we need to make to our model?
1. How can we share a weight matrix across multiple layers in PyTorch?
1. Write a module that predicts the third word given the previous two words of a sentence, without peeking.
1. What is a recurrent neural network?
1. What is "hidden state"?
1. What is the equivalent of hidden state in ` LMModel1`?
1. To maintain the state in an RNN, why is it important to pass the text to the model in order?
1. What is an "unrolled" representation of an RNN?
1. Why can maintaining the hidden state in an RNN lead to memory and performance problems? How do we fix this problem?
1. What is "BPTT"?
1. Write code to print out the first few batches of the validation set, including converting the token IDs back into English strings, as we showed for batches of IMDb data in <<chapter_nlp>>.
1. What does the `ModelResetter` callback do? Why do we need it?
1. What are the downsides of predicting just one output word for each three input words?
1. Why do we need a custom loss function for `LMModel4`?
1. Why is the training of `LMModel4` unstable?
1. In the unrolled representation, we can see that a recurrent neural network actually has many layers. So why do we need to stack RNNs to get better results?
1. Draw a representation of a stacked (multilayer) RNN.
1. Why should we get better results in an RNN if we call `detach` less often? Why might this not happen in practice with a simple RNN?
1. Why can a deep network result in very large or very small activations? Why does this matter?
1. In a computer's floating-point representation of numbers, which numbers are the most precise?
1. Why do vanishing gradients prevent training?
1. Why does it help to have two hidden states in the LSTM architecture? What is the purpose of each one?
1. What are these two states called in an LSTM?
1. What is tanh, and how is it related to sigmoid?
1. What is the purpose of this code in `LSTMCell`: `h = torch.cat([h, input], dim=1)`
1. What does `chunk` do in PyTorch?
1. Study the refactored version of `LSTMCell` carefully to ensure you understand how and why it does the same thing as the non-refactored version.
1. Why can we use a higher learning rate for `LMModel6`?
1. What are the three regularization techniques used in an AWD-LSTM model?
1. What is "dropout"?
1. Why do we scale the acitvations with dropout? Is this applied during training, inference, or both?
1. What is the purpose of this line from `Dropout`: `if not self.training: return x`
1. Experiment with `bernoulli_` to understand how it works.
1. How do you set your model in training mode in PyTorch? In evaluation mode?
1. Write the equation for activation regularization (in math or code, as you prefer). How is it different from weight decay?
1. Write the equation for temporal activation regularization (in math or code, as you prefer). Why wouldn't we use this for computer vision problems?
1. What is "weight tying" in a language model?

### Further Research

1. In ` LMModel2`, why can `forward` start with `h=0`? Why don't we need to say `h=torch.zeros(...)`?
1. Write the code for an LSTM from scratch (you may refer to <<lstm>>).
1. Search the internet for the GRU architecture and implement it from scratch, and try training a model. See if you can get results similar to those we saw in this chapter. Compare your results to the results of PyTorch's built in `GRU` module.
1. Take a look at the source code for AWD-LSTM in fastai, and try to map each of the lines of code to the concepts shown in this chapter.